In [11]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
# pyspark==2.4.7

'C:\\Spark\\spark-3.0.2-bin-hadoop2.7'

In [12]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [13]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.functions import sum,avg,max,min,mean
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window

In [14]:
ADMISSIONS      = spark.read.option("header",True).csv('C:\Health data\ADMISSIONS.csv')
CHARTEVENTS     = spark.read.option("header",True).csv('C:\Health data\CHARTEVENTS.csv')
DIAGNOSES_ICD   = spark.read.option("header",True).csv('C:\Health data\DIAGNOSES_ICD.csv')
D_ICD_DIAGNOSES = spark.read.option("header",True).csv('C:\Health data\D_ICD_DIAGNOSES.csv')
ICUSTAYS        = spark.read.option("header",True).csv('C:\Health data\ICUSTAYS.csv')
LABEVENTS       = spark.read.option("header",True).csv('C:\Health data\LABEVENTS.csv')
OUTPUTEVENTS    = spark.read.option("header",True).csv('C:\Health data\OUTPUTEVENTS.csv')
PATIENTS        = spark.read.option("header",True).csv('C:\Health data\PATIENTS.csv')

In [ ]:
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [8]:
# total_patients_num   = PATIENTS.select('SUBJECT_ID').distinct().count() # how many patients
total_Deceased_patients = PATIENTS.filter("EXPIRE_FLAG == '1'").drop('DOD_HOSP','DOD_SSN')
total_alive_patients    = PATIENTS.filter("EXPIRE_FLAG == '0'").drop('DOD_HOSP','DOD_SSN')

In [9]:
# Combine three database: LABEVENTS, OUTPUTEVENTS, and CHARTEVENTS ############  
LABEVENTS = LABEVENTS.select('SUBJECT_ID','ITEMID','CHARTTIME','VALUENUM').na.drop()
LABEVENTS = LABEVENTS.withColumnRenamed("VALUENUM", "VALUE")
OUTPUTEVENTS = OUTPUTEVENTS.select('SUBJECT_ID','ITEMID','CHARTTIME','VALUE').na.drop()
CHARTEVENTS = CHARTEVENTS.select('SUBJECT_ID','ITEMID','CHARTTIME','VALUENUM').na.drop()
CHARTEVENTS = CHARTEVENTS.withColumnRenamed("VALUENUM", "VALUE")

infor1 = OUTPUTEVENTS.union(LABEVENTS)
infor2 = infor1.union(CHARTEVENTS)
infor2 = infor2.withColumnRenamed("SUBJECT_ID", "ID")

In [10]:
# Re-map the feature ID ##############
Festure_ID = infor2.select('ITEMID').distinct()
w = Window().orderBy(lit('A'))
Festure_ID = Festure_ID.withColumn("rowNum", row_number().over(w)).withColumnRenamed("ITEMID", "ITEMID1") 
infor2 = infor2.join(Festure_ID, infor2.ITEMID==Festure_ID.ITEMID1,"left")\
               .select("ID","CHARTTIME","VALUE","rowNum")\
               .withColumnRenamed("rowNum", "ITEMID")

In [ ]:
# create SVMLight format for alive patients ###################
Alive_patients_infor = total_alive_patients.join(infor2, total_alive_patients.SUBJECT_ID == infor2.ID, "left")
Alive_patients_infor = Alive_patients_infor.withColumn("VALUE", Alive_patients_infor["VALUE"].cast("double"))
Alived = Alive_patients_infor.select('ID','ITEMID','VALUE').groupBy("ID","ITEMID").mean("VALUE")
Alived = Alived.withColumn("target", lit(0)).withColumnRenamed("avg(VALUE)", "VALUE")
Alived = Alived.filter(Alived.VALUE != 0)
Alived = Alived.withColumn("ITEMID", Alived["ITEMID"].cast("int"))

hu = Alived.select('ID').distinct()
w = Window().orderBy(lit('A'))
hu = hu.withColumn("rowNum", row_number().over(w)).withColumnRenamed("ID", "ID1") 
Alived_data = Alived.join(hu,Alived.ID==hu.ID1,"left")

Alived_data1 = Alived_data.filter(Alived_data.rowNum <= 10000)
Alived_data1 = Alived_data1.select('ID','ITEMID','target','VALUE')

Alived_data2 = Alived_data.filter( (Alived_data.rowNum <= 20000) & (Alived_data.rowNum > 10000) )
Alived_data2 = Alived_data2.select('ID','ITEMID','target','VALUE')

hu1 = Alived_data1.sort(Alived_data1.ID.asc(),Alived_data1.ITEMID.asc()).groupBy("ID", "target")\
    .agg(F.collect_list(F.struct("ITEMID", "VALUE")).alias("feature_value")) \
    .withColumn("feature_value", F.expr("transform(feature_value, x -> concat_ws(':',x.ITEMID, x.VALUE))")) \
    .withColumn("feature_value", F.concat_ws("  ", F.col("feature_value"))) \
    .withColumn("result", F.concat_ws(" ", F.col("Target"), F.col("feature_value"))) \
    .select("result")

hu1.coalesce(1).write.format("text").option("header", "false").mode("append").save("output_1.txt")

hu2 = Alived_data2.sort(Alived_data2.ID.asc(),Alived_data2.ITEMID.asc()).groupBy("ID", "target")\
    .agg(F.collect_list(F.struct("ITEMID", "VALUE")).alias("feature_value")) \
    .withColumn("feature_value", F.expr("transform(feature_value, x -> concat_ws(':',x.ITEMID, x.VALUE))")) \
    .withColumn("feature_value", F.concat_ws("  ", F.col("feature_value"))) \
    .withColumn("result", F.concat_ws(" ", F.col("Target"), F.col("feature_value"))) \
    .select("result")

hu2.coalesce(1).write.format("text").option("header", "false").mode("append").save("output_2.txt")

In [ ]:
# create SVMLight format for deceased patients ###################
Day_before_Deceased = 30
index_Date_Deceased = total_Deceased_patients.select('SUBJECT_ID','DOD').withColumn('index_date', F.date_sub('DOD', Day_before_Deceased))
index_Date = index_Date_Deceased.select('SUBJECT_ID','index_date')

infor3 = infor2.join(index_Date, infor2.ID ==  index_Date.SUBJECT_ID, "left")
infor4 = infor3.filter(infor3.CHARTTIME < infor3.index_date)

infor4 = infor4.withColumn("VALUE", infor4["VALUE"].cast("double")) 
Deceased = infor4.select('ID','ITEMID','VALUE').groupBy("ID","ITEMID").mean("VALUE")
Deceased = Deceased.withColumn("target", lit(1)).withColumnRenamed("avg(VALUE)", "VALUE")
Deceased = Deceased.filter(Deceased.VALUE != 0)
Deceased = Deceased.withColumn("ITEMID", Deceased["ITEMID"].cast("int"))

hu = Deceased.sort(Deceased.ID.asc(),Deceased.ITEMID.asc()).groupBy("ID", "target")\
    .agg(F.collect_list(F.struct("ITEMID", "VALUE")).alias("feature_value")) \
    .withColumn("feature_value", F.expr("transform(feature_value, x -> concat_ws(':',x.ITEMID, x.VALUE))")) \
    .withColumn("feature_value", F.concat_ws("  ", F.col("feature_value"))) \
    .withColumn("result", F.concat_ws(" ", F.col("Target"), F.col("feature_value"))) \
    .select("result")

hu.coalesce(1).write.format("text").option("header", "false").mode("append").save("output.txt")